# ECMWF Open Data 
---

Currently, open data are available from the following locations:
- [ECMWF Data Store](https://data.ecmwf.int/forecasts)

:::{figure} ../images/ecmwf-data-store.png
:label: fig:ecmwfds
:::

- [Amazon's AWS](https://aws.amazon.com/marketplace/pp/prodview-3ibagms7ky4ec?sr=0-670&ref_=beagle&applicationId=AWSMPContessa#usage)
:::{figure} ../images/aws.png
:label: fig:aws
:::

% - [Google Cloud](https://console.cloud.google.com/marketplace/product/bigquery-public-data/open-data-ecmwf)
% :::{figure} ../images/Google.png
% :label: fig:google
% :::
% :::{note}
% To access open data within Google Cloud, you will need to authenticate using your Google account or [`gsutil`](https://cloud.google.com/% storage/docs/gsutil) tool to access Cloud Storage from the command line.
% :::

- [Microsoft's Azure](https://planetarycomputer.microsoft.com/dataset/ecmwf-forecast)

:::{figure} ../images/Microsoft.png
:label: fig:microsoft
:::

In [18]:
import pystac_client
import planetary_computer
import requests
import xarray as xr

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["ecmwf-forecast"],
    query={
        "ecmwf:stream": {"eq": "oper"},
        "ecmwf:type": {"eq": "fc"},
        "ecmwf:step": {"eq": "0h"},
    },
)
items = search.item_collection()
len(items)

item = max(items, key=lambda item: item.datetime)
url = item.assets["data"].href
url = url.rpartition('?')[0]
print(url)
filename = str(url.rpartition('oper/')[-1])
print(filename)

r = requests.get(url, stream=True)
# with open(filename, mode="wb") as file:
#    for chunk in r.iter_content(chunk_size=10 * 1024):
#        file.write(chunk)
# ds = xr.open_dataset(filename, engine="cfgrib")
print("Cannot create file '20250531120000-0h-oper-fc.grib2.5b7b6.idx' Public access is not permitted on this storage account.")

https://ai4edataeuwest.blob.core.windows.net/ecmwf/20250531/12z/ifs/0p25/oper/20250531120000-0h-oper-fc.grib2
20250531120000-0h-oper-fc.grib2
Cannot create file '20250531120000-0h-oper-fc.grib2.5b7b6.idx' Public access is not permitted on this storage account.


- [open-meteo](https://open-meteo.com/en/docs/ecmwf-api)

:::{figure} ../images/open-meteo.png
:label: fig:openmeteo
:::

```{warning}
One can choose non-commercial, commercial or self-hosted licence. For non-commercial use, less than 10.000 daily API calls are available. For commercial use, different API plans and subscriptions are offered.
```

% - [Oikolab](https://docs.oikolab.com/#global-datasets) (only IFS open data)
% ```{warning}
% To retrieve open data, you will need to log into your account and then you will find your API key on the profile page.
% ```
% :::{figure} ../images/oikolab.png
% :label: fig:oikolab
% :::

In [ ]:
!pip3 install openmeteo-requests
!pip3 install requests-cache retry-requests numpy pandas

In [2]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 50,
	"longitude": 10,
	"hourly": ["temperature_2m", "temperature_2m_min", "temperature_2m_max"],
	"models": "ecmwf_ifs025",
	"timezone": "Europe/Berlin",
	"past_days": 92,
	"temporal_resolution": "hourly_3"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m_min = hourly.Variables(1).ValuesAsNumpy()
hourly_temperature_2m_max = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["temperature_2m_min"] = hourly_temperature_2m_min
hourly_data["temperature_2m_max"] = hourly_temperature_2m_max

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 50.0°N 10.0°E
Elevation 291.0 m asl
Timezone b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0 7200 s
                         date  temperature_2m  temperature_2m_min  \
0   2025-02-28 22:00:00+00:00             NaN                 NaN   
1   2025-03-01 01:00:00+00:00             NaN                 NaN   
2   2025-03-01 04:00:00+00:00             NaN                 NaN   
3   2025-03-01 07:00:00+00:00             NaN                 NaN   
4   2025-03-01 10:00:00+00:00             NaN                 NaN   
..                        ...             ...                 ...   
787 2025-06-07 07:00:00+00:00       16.291500             14.8915   
788 2025-06-07 10:00:00+00:00       18.541500                 NaN   
789 2025-06-07 13:00:00+00:00       20.191500                 NaN   
790 2025-06-07 16:00:00+00:00       19.741499                 NaN   
791 2025-06-07 19:00:00+00:00       18.391500                 NaN   

     temperature_2m_max  
0                   NaN  
1

:::{seealso}
For a full list of other locations, where you can find open data available, visit the [ECMWF open datasets](https://confluence.ecmwf.int/display/DAC/ECMWF+open+data%3A+real-time+forecasts+from+IFS+and+AIFS) website.
:::

:::{card}
1. Copyright Statement: Copyright "© 2025 European Centre for Medium-Range Weather Forecasts (ECMWF)".

2. Source: www.ecmwf.int

3. Licence Statement: This data is published under a Creative Commons Attribution 4.0 International (CC BY 4.0). https://creativecommons.org/licenses/by/4.0/

4. Disclaimer: ECMWF does not accept any liability whatsoever for any error or omission in the data, their availability, or for any loss or damage arising from their use.
:::